In [1]:
pip install xlwings

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import pandas as pd
import xlwings as xw

PATH = "SALARY_NEW.xls"
wb = xw.Book(PATH)
sheet = wb.sheets['DATA']

df = sheet['B:M'].options(pd.DataFrame, index=False, header=True).value
df = df.dropna(how="all")


In [14]:
entities = df.FOS.unique()
products = ["PL SAL", "PL SELF", "DIGITAL"]
bkt_types = ["FE", 1, 2, 3,4]
column_sum={}
column_sum2={}
enity_sum={}
enity_sum2 = {}
dfs = {}
dfs2 = {}
dfs3 = {}
for entity in entities:
    column_sum ={}
    column_sum2 = {}
    for bkt in bkt_types:
        for product in products:
            key = f"df_{entity}_{product}_{bkt}"
            dfs[key] = df.loc[(df['FOS'] == entity) & (df['PRODUCT'] == product) & (df['BKT'] == bkt) & (df['Status'] == 'PAID')]
            dfs2[key] = df.loc[(df['FOS'] == entity) & (df['PRODUCT'] == product) & (df['BKT'] == bkt) & (df['Status'] == 'UNPAID')]
            dfs3[key] = df.loc[(df['FOS'] == entity) & (df['PRODUCT'] == product) & (df['BKT'] == bkt) & (df['RB'] == 'NM')]
            condition = dfs[key]['POS'].sum()
            condition2 = dfs2[key]['POS'].sum()
            condition3 = dfs3[key]['ACTUAL EMI'].sum()
            total = condition + condition2
            result = (condition/total) if total != 0 else 0
            nm_result = (condition3/total) if total != 0 else 0           
            column_sum[key]=result
            column_sum2[key]= nm_result
    enity_sum[entity]  = column_sum
    enity_sum2[entity]  = column_sum2


In [15]:
writer = pd.ExcelWriter("Refine2.xlsx", engine="xlsxwriter")
workbook = writer.book
worksheet = workbook.add_worksheet()
merge_format = workbook.add_format({'align': 'center', 'valign': 'vcenter', 'border': 1,'bold' : True})
worksheet.merge_range(17, 0, 17, 13, 'DEC23', merge_format)

start_row = 3  

for j, key in enumerate(enity_sum):
    worksheet.write(start_row+j, 0, key, merge_format)
    for i, key_sum in enumerate(enity_sum[key]):
       worksheet.write(3+j, 1+i, enity_sum[key][key_sum])

start_r = 15
    
for j, key in enumerate(enity_sum2):
    worksheet.write(start_r+j, 0, key, merge_format)
    for i, key_sum in enumerate(enity_sum2[key]):
       worksheet.write(15+j, 15+i, enity_sum2[key][key_sum])

 
writer.save()
writer.close()


C:\Users\aviri\AppData\Local\Temp\ipykernel_27436\3579046655.py:22: FutureWarning: save is not part of the public API, usage can give in unexpected results and will be removed in a future version
  writer.save()
c:\Users\aviri\AppData\Local\Programs\Python\Python310\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [16]:
sheet2 = wb.sheets['Payout']
df2 = sheet2['A:I'].options(pd.DataFrame, index=False, header=True).value
df2 = df2.dropna(how="all")

df_FE = df2.iloc[1:3 ,1:6]
df_FE.columns = df_FE.iloc[0]
df_FE = df_FE[1:]

df_bkt1_PLSAL = df2.iloc[4:10 ,0:6]
df_bkt1_PLSAL.columns = df_bkt1_PLSAL.iloc[0]
df_bkt1_PLSAL = df_bkt1_PLSAL[1:]
df_bkt1_PLSAL.set_index('NM', inplace=True)

df_bkt1_PLSELF = df2.iloc[11:16 ,0:6]
df_bkt1_PLSELF.columns = df_bkt1_PLSELF.iloc[0]
df_bkt1_PLSELF = df_bkt1_PLSELF[1:]
df_bkt1_PLSELF.set_index('NM', inplace=True)

df_bkt2 = df2.iloc[17:21 ,0:5]
df_bkt2.columns = df_bkt2.iloc[0]
df_bkt2 = df_bkt2[1:]
df_bkt2.set_index('NM', inplace=True)

df_bkt4 = df2.iloc[22:26 ,0:5]
df_bkt4.columns = df_bkt4.iloc[0]
df_bkt4 = df_bkt4[1:]
df_bkt4.set_index('NM', inplace=True)



In [17]:
def bkt_name(key):
    a = key.split('_')
    bkt_name = a[3]
    if a[3] == "1":
        bkt_name = a[3]+"_"+a[2] 
    return bkt_name 

def get_index_normal(df,v1):
    index_list = list(df.columns)
    
    for i,index in enumerate(index_list):
        if index>v1:
            to_return_upper = index_list[i-1]
            break
        elif index==v1:
            to_return_upper = index
            break
        elif v1 == 0:
            to_return_upper = 0
            break
    return to_return_upper

def get_index_nm(df,v2):
    
    index_list = df['NM'].tolist()
    
    for i,index in enumerate(index_list):
        if index>v2:
            to_return_side = index_list[i-1]
            break
        elif index==v2:
            to_return_side = index
            break
        elif v2 == 0:
            to_return_side = 0
            break
    return to_return_side  

  

In [18]:
Dict1 = {'FE': df_FE, '1_PL SAL': df_bkt1_PLSAL, '1_PL SELF':df_bkt1_PLSELF, '2':df_bkt2, '3':df_bkt2, '4':df_bkt4}


'''enity_sum ={"Sudesh_bkt1_PLSAL":"test"}
for i,key in enumerate(enity_sum): 
    bkt_name   =  bkt_name(key)
v1 = 0.84  
index = get_index(Dict1[bkt_name],v1)'''

        

'enity_sum ={"Sudesh_bkt1_PLSAL":"test"}\nfor i,key in enumerate(enity_sum): \n    bkt_name   =  bkt_name(key)\nv1 = 0.84  \nindex = get_index(Dict1[bkt_name],v1)'

In [19]:
for key in enity_sum:
    for key2 in enity_sum[key]:
        v1 = enity_sum[key][key2]
        v2 = enity_sum2[key][key2]
        bkt_name_final = bkt_name(key2)
        index1 = get_index_normal(Dict1[bkt_name_final],v1)
        print(index1)
        if (bkt_name_final != "FE" and v2 !=0) :
            index2 = get_index_nm(Dict1[bkt_name_final],v2)
            print(index2)
            
        print ("____________________")

            

           
        
print(index2)

0.88
____________________
0.88
____________________
1.0
____________________
0.85


KeyError: 'NM'

In [ ]:
df3.loc[0.08,0.8]

0.03